<a href="https://colab.research.google.com/github/alexzhang-aero/vtol-path-planning/blob/main/VTOL_Path_Planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VTOL Flight Plan - National Weather Center to Oklahoma State University


In [ ]:
!pip install pydeck

import pydeck as pdk
import pandas as pd
import numpy as np
import sys
from math import cos, sin, asin, sqrt, pi

#-------------------------------------------------------------------------
# Plotting known mesonet sites in the region
#-------------------------------------------------------------------------

# Load CSV into pandas dataframe
data = pd.read_csv('https://gist.githubusercontent.com/alexzhang-aero/a673b78c0ea918e906435bbcc059d8b1'
                  '/raw/c132ad315cefe67368cade837002f0d151983794/geoinfo.csv')
df = pd.DataFrame(data, columns = ['nlat', 'elon', 'stid'])
print(df)

layer = pdk.Layer(
    'ScatterplotLayer',
    df,
    pickable=True,
    opacity=1,
    stroked=True,
    filled=True,
    radius_scale=6,
    radius_min_pixels=2,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position=['elon', 'nlat'],
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

# Set the viewport location
view_state = pdk.ViewState(latitude=35.4676, longitude=-97.5164, zoom=7, bearing=0, pitch=0)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={'text': '{stid}'})
r.to_html('scatterplot_layer.html')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
        nlat      elon  stid
0   34.80833 -98.02325  ACME
1   35.17156 -96.63121  BOWL
2   35.78050 -96.35404  BRIS
3   36.14730 -97.28585  CARL
4   35.65282 -96.80407  CHAN
5   35.03236 -97.91446  CHIC
6   35.54848 -98.03654  ELRE
7   35.84891 -97.47978  GUTH
8   35.85431 -97.95442  KIN2
9   35.88050 -97.91121  KING
10  36.06434 -97.21271  MARE
11  36.11860 -97.60140  MARS
12  35.27225 -97.95553  MINC
13  36.11685 -97.60685  MRSH
14  34.96774 -97.95202  NINN
15  35.25560 -97.48360  NORM
16  35.18160 -97.43980  NRMN
17  36.03126 -96.49749  OILT
18  35.47226 -97.46414  OKCE
19  35.55560 -97.51070  OKCN
20  35.47112 -97.58245  OKCW
21  35.43172 -96.26265  OKEM
22  35.99865 -97.04831  PERK
23  35.17660 -96.70028  SEMI
24  35.36492 -96.94822  SHAW
25  35.54208 -97.34146  SPEN
26  36.12093 -97.09527  STIL
27  34.98224 -97.52109  WASH
28  35.55671 -97.75538  YUKO


<IPython.core.display.Javascript object>

In [ ]:
#-------------------------------------------------------------------------
# Choosing Mesonet Waypoints: Dijkstra's Shortest Path
#-------------------------------------------------------------------------

# Create graph class
class Graph(object):
  def __init__(self, nodes, init_graph):
    self.nodes = nodes
    self.graph = self.construct_graph(nodes, init_graph)

  def construct_graph(self, nodes, init_graph):
    graph = {}
    for node in nodes:
      graph[node] = {}
    graph.update(init_graph)
    return graph

  # returns nodes
  def get_nodes(self):
    return self.nodes

  # returns node neighbors
  def get_outgoing_edges(self, node):
    connections = []
    for out_node in self.nodes:
      if self.graph[node].get(out_node, False) != False:
        connections.append(out_node)
    return connections

  # add weighted edge
  def add_edge(node1, node2, weight):
    init_graph[node1][node2] = weight

  # returns edge value
  def value(self, node1, node2):
    return self.graph[node1][node2]

def dijkstra_shortest_path(graph, start_node):
  unvisited_nodes = list(graph.get_nodes())

  # stores best known cost of visiting each node
  shortest_path = {}
  # stores shortest known path to a node
  previous_nodes = {}

  # set unvisited nodes to value of "infinity"
  max_value = sys.maxsize
  for node in unvisited_nodes:
    shortest_path[node] = max_value
  # set starting node value to 0
  shortest_path[start_node] = 0

  while unvisited_nodes:
    current_min_node = None
    # find node with lowest score
    for node in unvisited_nodes:
      if current_min_node == None:
        current_min_node = node
      elif shortest_path[node] < shortest_path[current_min_node]:
        current_min_node = node

    # retrieve current nodes neighbors and update distances
    neighbors = graph.get_outgoing_edges(current_min_node)
    for neighbor in neighbors:
      tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
      if tentative_value < shortest_path[neighbor]:
        shortest_path[neighbor] = tentative_value
        # update best path to current node
        previous_nodes[neighbor] = current_min_node

    # mark node as visited
    unvisited_nodes.remove(current_min_node)

  return previous_nodes, shortest_path

def print_result(previous_nodes, shortest_path, start_node, target_node):
  path = []
  node = target_node
  while node != start_node:
    path.append(node)
    node = previous_nodes[node]
  path.append(start_node)
  path.reverse()
  print("Total distance: {} ".format(shortest_path[target_node]) + "miles")
  print(" -> ".join(path))
  return path

# Distance between coordinates in miles using Haversine formula
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 3956 * 2 * asin(sqrt(a))

# Driver code

# Plan to utilize 80% of the VTOL's max battery life
# Assume perfect flying conditions: no wind or inclement weather
battery_util = 0.80
speed = 45 #ft/s
max_battery_life = 60 #mins
max_flight_time = battery_util * max_battery_life * 60 #seconds
max_flight_dist = (max_flight_time * speed) / 5280 #mi

# initialize nodes
nodes = df['stid'].to_numpy()

# initialize weighted edges
init_graph = {}
for node in nodes:
  init_graph[node] = {}
  for ii in range(nodes.size):
    if nodes[ii] != node:
      node_idx = df.index[df['stid'] == node].tolist()
      lat1, lon1 = df.loc[node_idx[0], 'nlat': 'elon']
      lat2, lon2 = df.loc[ii, 'nlat': 'elon']
      weight = distance(lat1, lon1, lat2, lon2)
      # only create edge if distance < max flight distance
      if weight <= max_flight_dist:
        Graph.add_edge(node, nodes[ii], weight)

graph = Graph(nodes, init_graph)
start_node = 'NRMN' # NATIONAL WEATHER CENTER
target_node = 'STIL' # OKLAHOMA STATE UNIVERSITY
previous_nodes, shortest_path = dijkstra_shortest_path(graph, start_node)
waypointNames = print_result(previous_nodes, shortest_path, start_node, target_node)

# get coordinate waypoints for shortest path
waypoints = []
for node in waypointNames:
  node_idx = df.index[df['stid'] == node].tolist()
  lat, lon = df.loc[node_idx[0], 'nlat': 'elon']
  coord = [lat, lon]
  waypoints.append(coord)

df2 = pd.DataFrame(data = waypoints, columns=['lat', 'lon'])

layer = pdk.Layer(
    'ScatterplotLayer',
    data=df2,
    pickable=True,
    opacity=1,
    stroked=True,
    filled=True,
    radius_scale=6,
    radius_min_pixels=2,
    radius_max_pixels=100,
    line_width_min_pixels=1,
    get_position=['lon', 'lat'],
    get_fill_color=[255, 140, 0],
    get_line_color=[0, 0, 0],
)

# Set the viewport location and render
view_state = pdk.ViewState(latitude=35.4676, longitude=-97.5164, zoom=7, bearing=0, pitch=0)
r = pdk.Deck(layers=[layer], initial_view_state=view_state, tooltip={'text': '{lat}, {lon}'})
r.to_html('scatterplot_layer.html')

Total distance: 75.46371013283297 miles
NRMN -> OKCE -> OKCN -> GUTH -> MARE -> STIL


<IPython.core.display.Javascript object>